In [9]:
PROJECT_PATH = "/Users/stephenbo/JOB/PROJECT/quantification_new/"
import sys, os
sys.path.append(PROJECT_PATH + "./utils")
import mysql as mq 
from tools import *
import time
from joinquant import *
import pandas as pd

In [2]:
mysql = mq.MY_SQL()
jqdata = joinquant("account6")

auth success 
login account=18909717222 success.


In [3]:
select_sql = open("/Users/stephenbo/JOB/PROJECT/quantification_new/sql/get_stock_info.sql").read()
print(select_sql)

SELECT  t.stock_name AS stock_name 
        ,MAX(t.stock_code) AS stock_code 
        ,MAX(t.start_date) AS start_date 
        ,MAX(t.end_date) AS end_date
FROM 
(
    SELECT  t1.stock_name AS stock_name 
            ,t1.stock_code AS stock_code 
            ,CASE WHEN t2.stock_code IS NULL THEN t1.start_date ELSE t2.ds END AS start_date 
            ,t1.end_date AS end_date 
    FROM 
    (
        SELECT  stock_name 
                ,MAX(stock_code) AS stock_code  
                ,MAX(start_date) AS start_date  
                ,MAX(end_date) AS end_date  
        FROM    stock_info 
        GROUP BY stock_name 
    ) t1
    LEFT OUTER JOIN 
    (
        SELECT  stock_code 
                ,MAX(ds) AS ds 
        FROM    stock_daily_price_tmp 
        GROUP BY stock_code 
    ) t2 ON t1.stock_code = t2.stock_code 
) t
GROUP BY t.stock_name
;


In [4]:
stock_info = mysql.select_table(select_sql)

In [5]:
mysql.create_table(
    table_name = "stock_daily_price_tmp"#"stock_daily_price"
    ,table_elements = {
        "stock_code":       "VARCHAR(1000)"
        ,"ds":              "VARCHAR(1000)"
        ,"open":            "FLOAT"
        ,"close":           "FLOAT"
        ,"high":            "FLOAT"
        ,"low":             "FLOAT"
        ,"volume":          "FLOAT"
        ,"money":           "FLOAT"
    }
)

In [6]:
stock_info[:4]

(('平安银行', '000001.XSHE', '2021-12-16', '2200-01-01'),
 ('万科A', '000002.XSHE', '2021-12-16', '2200-01-01'),
 ('国华网安', '000004.XSHE', '2021-12-16', '2200-01-01'),
 ('ST星源', '000005.XSHE', '2021-12-16', '2200-01-01'))

In [7]:
stock_start_date_dict = dict()
stock_end_date_dict = dict()
stock_code_list = list()
for element in stock_info:
    stock_code = element[1]
    start_date = element[2]
    end_date = element[3]
    stock_start_date_dict[stock_code] = start_date 
    stock_end_date_dict[stock_code] = end_date 
    stock_code_list.append(stock_code)

In [8]:
stock_daily_price = None
for stock_code in stock_code_list:
    start_date = string_date_convert(stock_start_date_dict[stock_code])
    end_date = "2021-12-16 23:59:59"#get_current_time()
    if start_date != "2021-12-16 00:00:00":
        print("Processing stock_code=%s;start_date=%s;end_date=%s ..." % (stock_code, start_date, end_date))
        stock_daily_price = jqdata.get_daily_price(
            stock_code
            ,start_date = start_date
            ,end_date = end_date
        )
        stock_daily_price['open'] = stock_daily_price['open'].fillna(-1.)
        stock_daily_price['close'] = stock_daily_price['close'].fillna(-1.)
        stock_daily_price['high'] = stock_daily_price['high'].fillna(-1.)
        stock_daily_price['low'] = stock_daily_price['low'].fillna(-1.)
        stock_daily_price['volume'] = stock_daily_price['volume'].fillna(-1.)
        stock_daily_price['money'] = stock_daily_price['money'].fillna(-1.)
        stock_daily_price_elements = mysql.get_insert_elements(stock_daily_price, stock_daily_price.columns)
        mysql.insert_table(
            table_name = "stock_daily_price_tmp"
            ,attribute_list = stock_daily_price.columns
            ,elements = stock_daily_price_elements
        )
        time.sleep(1)
stock_daily_price

Processing stock_code=600572.XSHG;start_date=2004-04-12 00:00:00;end_date=2021-12-16 23:59:59 ...


/Users/stephenbo/opt/anaconda3/envs/quant/lib/python3.8/site-packages/jqdatasdk/api.py:34: PanelObsoleteWarning: 当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口的 panel 参数将固定为 False（0.25 及以上版本的 pandas 不再支持 panel，如使用该数据结构和相关函数请注意修改）
  warnings.warn(PandasChecker.VERSION_NOTICE_MESSAGE, PanelObsoleteWarning)


Exception: 您当天的查询条数超过了每日最大查询限制：100万条；付费可增加流量权限，详情请咨询管理员，微信号：JQData01

In [10]:
jqdata.get_daily_price(
        "000002.XSHE"
        ,start_date = "2021-12-18 00:00:00"
        ,end_date = "2021-12-19 00:27:00"
    )

,ds,open,close,high,low,volume,money,stock_code
